# Classification of Compliance in Property Maintenance Files

This project was completed as part of the "[Applied Machine Learning in Python](https://www.coursera.org/learn/python-machine-learning)" course by the University of Michigan on Coursera.

The dataset uploaded with this file was provided during this course and is based on a Kaggle competition (which includes the datasets) found here: [https://www.kaggle.com/competitions/detroit-blight-ticket-compliance/data](https://www.kaggle.com/competitions/detroit-blight-ticket-compliance/data).

This dataset is about blight tickets issued in the Detroit area, showing the compliance to paying these tickets. The goal is to develop a classification model that returns the probability for future cases to be compliant. The cases to be returned are provided in the 'test.csv' file and the training data is provided in the 'train.csv' file. Additionally there are two more .csv files containing information about addresses and their latitude and longitudonal coordinates.

This notebook goes through the whole process of:
* Processing and cleaning the data
* Preparing the data for classification
* Training the classification model with GridSearch to find optimal hyperparameters
* Predicting the test data

The class is set to either (0 for not compliant), (1 for compliant) or (NaN for not responsible) in the "compliance" feature of the train data set.

All of the features in the train and test datasets are shown below.

<br>

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant

In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

Define a function used to show if the data is complete or if any values are still missing for the analysis

In [21]:
def give_update(df):
    total_length = len(df)
    for column in df.columns:
        print(f'{column}: {df[column].isnull().sum()}/{total_length}')

### Import the data

In [22]:
df_train = pd.read_csv('readonly/train.csv', encoding = "ISO-8859-1", low_memory=False) # The training set
df_test = pd.read_csv('readonly/test.csv') # The testing set (does not contain information on the class)
df_address = pd.read_csv('readonly/addresses.csv')
df_latlon = pd.read_csv('readonly/latlons.csv')

First I will look at the data to see how many datapoints are included in each df and what the dtype of each column is, as well as what columns are included in each df.

In [23]:
print(df_train.info())
print(df_test.info())
print(df_address.info())
print(df_latlon.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ticket_id                   250306 non-null  int64  
 1   agency_name                 250306 non-null  object 
 2   inspector_name              250306 non-null  object 
 3   violator_name               250272 non-null  object 
 4   violation_street_number     250306 non-null  float64
 5   violation_street_name       250306 non-null  object 
 6   violation_zip_code          0 non-null       float64
 7   mailing_address_str_number  246704 non-null  float64
 8   mailing_address_str_name    250302 non-null  object 
 9   city                        250306 non-null  object 
 10  state                       250213 non-null  object 
 11  zip_code                    250305 non-null  object 
 12  non_us_str_code             3 non-null       object 
 13  country       

So I can see that the train dataset contains 34 columns and 250305 entries. The test dataset only contains 27 columns, which means I need to check which columns will not be available when classifying the test set to not train the model on non-existing data.
The address dataset is complete and the longitute and latitude dataset is only missing 7 values.

Additionally I noticed that there were 3 entries for 'non_us_str_code', which means I will check if there are any entries in the test set outside the USA or if these entries can be discarded.

In [24]:
print(len(df_train[df_train['country']!='USA']))
print(len(df_test[df_test['country']!='USA']))
print(df_train[df_train['country']!='USA'])
print(df_test.loc[df_test['country'].isna(),['city','country']])

13
0
        ticket_id                                     agency_name  \
160652     189866                      Department of Public Works   
177864     209821                      Department of Public Works   
178520     209675                      Department of Public Works   
211755     245108  Buildings, Safety Engineering & Env Department   
216567     250642                      Department of Public Works   
216568     250643                      Department of Public Works   
216927     250552                      Department of Public Works   
217529     251516                      Department of Public Works   
222230     256184                      Department of Public Works   
226259     259979                      Department of Public Works   
226609     259990                      Department of Public Works   
226610     259991                      Department of Public Works   
236075     270253                      Department of Public Works   

            inspector_name  

So it can be seen that there are 13 entries in the train set which are outside the USA, but none in the test set. This means these testcases are most likely special cases and I will not consider them further in the test set. 
This means I will drop all the cases with countries outside the USA as well as the test cases which show the person to not be responsible (i.e. compliance=NaN).

In [25]:
# process data
df_train = df_train[df_train['country']=='USA'] # Drop all entries that are not from the US
df_train = df_train.dropna(subset=['compliance']) # Drop all NaNs in compliance
df_train.drop(['country'], axis=1, inplace=True) # Can be dropped, since is all USA now
df_test.drop(['country'], axis=1, inplace=True) # Can be dropped, since is all USA now

I will now inspect the columns which are not included in the test set to see if they contain any information that could be used to fill any of the NaN values in the dataframe.

In [26]:
print(df_train[[column for column in df_train.columns if column not in np.append(df_test.columns, 'compliance')]].head())

   payment_amount  balance_due         payment_date      payment_status  \
0             0.0        305.0                  NaN  NO PAYMENT APPLIED   
1           780.0         75.0  2005-06-02 00:00:00        PAID IN FULL   
5             0.0        305.0                  NaN  NO PAYMENT APPLIED   
6             0.0        855.0                  NaN  NO PAYMENT APPLIED   
7             0.0        140.0                  NaN  NO PAYMENT APPLIED   

  collection_status                         compliance_detail  
0               NaN               non-compliant by no payment  
1               NaN  compliant by late payment within 1 month  
5               NaN               non-compliant by no payment  
6               NaN               non-compliant by no payment  
7               NaN               non-compliant by no payment  


It can be seen that the only columns that are not contained in the test set but indeed in the training set are related to the payment/compliance of the ticket. This means all of these columns can be dropped, as they do not contain any useful information for filling NaN values and are not available for prediction later on.
Additionally the address and longitude and latitude data sets are merged with the trianing and testing set.

In [27]:
# drop all columns not available in test
df_train.drop([column for column in df_train.columns if column not in np.append(df_test.columns, 'compliance')], axis=1, inplace=True)

# merge addresse information
df_ad_ll = pd.merge(df_address, df_latlon, on='address', how='left')
df_train = pd.merge(df_train, df_ad_ll, how='left', on='ticket_id')
df_test = pd.merge(df_test, df_ad_ll, how='left', on='ticket_id')

At this point it would be helpful to look again at how many NaN values are in each of the two train and test sets. For this I will call the give_update() function defined above.

In [28]:
give_update(df_train)
print('\n')
give_update(df_test)

ticket_id: 0/159869
agency_name: 0/159869
inspector_name: 0/159869
violator_name: 26/159869
violation_street_number: 0/159869
violation_street_name: 0/159869
violation_zip_code: 159869/159869
mailing_address_str_number: 2556/159869
mailing_address_str_name: 3/159869
city: 0/159869
state: 80/159869
zip_code: 1/159869
non_us_str_code: 159869/159869
ticket_issued_date: 0/159869
hearing_date: 227/159869
violation_code: 0/159869
violation_description: 0/159869
disposition: 0/159869
fine_amount: 0/159869
admin_fee: 0/159869
state_fee: 0/159869
late_fee: 0/159869
discount_amount: 0/159869
clean_up_cost: 0/159869
judgment_amount: 0/159869
grafitti_status: 159869/159869
compliance: 0/159869
address: 0/159869
lat: 2/159869
lon: 2/159869


ticket_id: 0/61001
agency_name: 0/61001
inspector_name: 0/61001
violator_name: 28/61001
violation_street_number: 0/61001
violation_street_name: 0/61001
violation_zip_code: 36977/61001
mailing_address_str_number: 1014/61001
mailing_address_str_name: 3/61001
city

The first thing to notice is that the 'graffiti_status' and 'violation_zip_code' contains only NaN values in the train set, which means this column can be dropped.

The addresses will be inspected further to see if information might be included as a duplicate.

In [29]:
print(df_train[['address', 'zip_code','violation_street_number','violation_street_name','violation_zip_code','mailing_address_str_number','mailing_address_str_name']])

                             address   zip_code  violation_street_number  \
0             2900 tyler, Detroit MI      60606                   2900.0   
1           4311 central, Detroit MI      48208                   4311.0   
2        6478 northfield, Detroit MI  908041512                   6478.0   
3         8027 brentwood, Detroit MI      48038                   8027.0   
4        8228 mt elliott, Detroit MI      48211                   8228.0   
...                              ...        ...                      ...   
159864   20009 northlawn, Detroit MI      48235                  20009.0   
159865       15725 steel, Detroit MI      48227                  15725.0   
159866      7152 chicago, Detroit MI      48204                   7152.0   
159867  17403 mt elliott, Detroit MI      48212                  17403.0   
159868      15634 novara, Detroit MI      95926                  15634.0   

       violation_street_name  violation_zip_code  mailing_address_str_number  \
0      

It can be seen that the 'addresse' column is the same as the 'violator_...' columns. This means the violator columns will be dropped. Additionally the columns relating to the mailing addresse will be dropped as well, apart from the zip-code, as otherwise too much information about this is contained in the data set and the zip code is already in numerical format. It needs to be verified however that the zip code only has length of 5.

First I try to clean up the zip-code column with the city names. This works well for the trianing set. However with teh test set I notice some problems, as the three entries with missing zip-code are in fact not in the USA after all. These three entries will be classified as non-compliant and will not be predicted with the classifier.

In [30]:
for index, row in df_train[df_train['zip_code'].isnull()].iterrows():
    estimated = df_train.loc[df_train['city']==row['city'],'zip_code'].dropna()
    if len(estimated)!=0:
        df_train.loc[index,'zip_code'] = estimated.iloc[0]

list_non_compliant_test = []
for index, row in df_test[df_test['zip_code'].isnull()].iterrows():
    list_non_compliant_test.append(row['ticket_id']) # these will all get a score of 0 in the final prediction

df_test = df_test[~df_test['ticket_id'].isin(list_non_compliant_test)]

Then I clean up the lat and lon columns using the violation street name, before dropping all of the unnecessary columns.
If I cannot find a corresponding street name, I will take the mean of the whole dataset to fill the NaN.

In [31]:
# fill lat and lon with mean of same city 

for index, row in df_train[df_train['lat'].isnull()].iterrows():
    estimated = df_train.loc[df_train['violation_street_name'].str.lower()==row['violation_street_name'].lower(),['lat','lon']].dropna()
    if len(estimated)==0:
        df_train.loc[index,'lat'] = df_train['lat'].dropna().mean()
        df_train.loc[index,'lon'] = df_train['lon'].dropna().mean()
    else:
        df_train.loc[index,'lat'] = estimated['lat'].mean()
        df_train.loc[index,'lon'] = estimated['lon'].mean()

for index, row in df_test[df_test['lat'].isnull()].iterrows():
    estimated = df_test.loc[df_test['violation_street_name'].str.lower()==row['violation_street_name'].lower(),['lat','lon']].dropna()
    if len(estimated)==0:
        df_test.loc[index,'lat'] = df_train['lat'].dropna().mean()
        df_test.loc[index,'lon'] = df_train['lon'].dropna().mean()
    else:
        df_test.loc[index,'lat'] = estimated['lat'].mean()
        df_test.loc[index,'lon'] = estimated['lon'].mean()

Check for any zip codes longer than 5 digits. If cannot find substitute, drop them from train set or add them to non compliant list in test set.

In [32]:
for index, row in df_train.loc[df_train['zip_code'].str.len() > 5].iterrows():
    estimated = df_train.loc[df_train['city']==row['city'],'zip_code'].dropna()
    if len(estimated)!=0:
        df_train.loc[index,'zip_code'] = estimated.iloc[0]

for index, row in df_train.loc[df_train['zip_code'].str.len() > 5].iterrows():
    estimated = df_train.loc[df_train['mailing_address_str_name']==row['mailing_address_str_name'],'zip_code'].dropna()
    if len(estimated)!=0:
        df_train.loc[index,'zip_code'] = estimated.iloc[0]

df_train = df_train.loc[df_train['zip_code'].str.len() <= 5]
df_train['zip_code'] = pd.to_numeric(df_train['zip_code'], errors='coerce')
df_train = df_train[~df_train['zip_code'].isna()]

for index, row in df_test.loc[df_test['zip_code'].str.len() > 5].iterrows():
    estimated = df_train.loc[df_train['city']==row['city'],'zip_code'].dropna()
    if len(estimated)!=0:
        df_test.loc[index,'zip_code'] = estimated.iloc[0]

for index, row in df_test.loc[df_test['zip_code'].str.len() > 5].iterrows():
    estimated = df_train.loc[df_train['mailing_address_str_name']==row['mailing_address_str_name'],'zip_code'].dropna()
    if len(estimated)!=0:
        df_test.loc[index,'zip_code'] = estimated.iloc[0]
    else:
        list_non_compliant_test.append(row['ticket_id'])

df_test['zip_code'] = pd.to_numeric(df_test['zip_code'], errors='coerce')
list_non_compliant_test.extend(df_test.loc[df_test['zip_code'].isna(),'ticket_id'].to_list())
df_test = df_test[~df_test['ticket_id'].isin(list_non_compliant_test)]
df_test = df_test[~df_test['zip_code'].isna()]


In [33]:
to_drop = ['address',
            'city',
            'violation_street_number',
            'violation_street_name',
            'violation_zip_code',
            'mailing_address_str_number',
            'mailing_address_str_name',
            'violator_name',
            'non_us_str_code',
            'violation_description',
            'grafitti_status',
            'state']
df_train.drop(to_drop, axis=1, inplace=True)
df_test.drop(to_drop, axis=1, inplace=True)

In [34]:
give_update(df_train)
print('\n')
give_update(df_test)
print(df_train.dtypes)

ticket_id: 0/159833
agency_name: 0/159833
inspector_name: 0/159833
zip_code: 0/159833
ticket_issued_date: 0/159833
hearing_date: 227/159833
violation_code: 0/159833
disposition: 0/159833
fine_amount: 0/159833
admin_fee: 0/159833
state_fee: 0/159833
late_fee: 0/159833
discount_amount: 0/159833
clean_up_cost: 0/159833
judgment_amount: 0/159833
compliance: 0/159833
lat: 0/159833
lon: 0/159833


ticket_id: 0/60873
agency_name: 0/60873
inspector_name: 0/60873
zip_code: 0/60873
ticket_issued_date: 0/60873
hearing_date: 2197/60873
violation_code: 0/60873
disposition: 0/60873
fine_amount: 0/60873
admin_fee: 0/60873
state_fee: 0/60873
late_fee: 0/60873
discount_amount: 0/60873
clean_up_cost: 0/60873
judgment_amount: 0/60873
lat: 0/60873
lon: 0/60873
ticket_id               int64
agency_name            object
inspector_name         object
zip_code                int64
ticket_issued_date     object
hearing_date           object
violation_code         object
disposition            object
fine_amou

At this point I dropped all of the unnecessary columns and I only have informative columns left and no duplicate information.
The only data that is left with NaN values is the hearing date column.
For this column I will calculate the average time it takes from the ticket being issued and the hearing.
Additionally I will create a new column which contains exactly this time difference and I will drop the two date columns.

Additionally I will convert all remaining string columns to lowercase letter.

In [35]:
# convert strings to lowercase letters
to_lower = ['agency_name', 'inspector_name', 'disposition']
for name in to_lower:
    df_train[name] = df_train[name].str.lower()
    df_test[name] = df_test[name].str.lower()
    
# Calculate the gap between hearing date and ticket issued date and convert it to int
df_train[['ticket_issued_date','hearing_date']] = df_train[['ticket_issued_date','hearing_date']].apply(pd.to_datetime)
df_test[['ticket_issued_date','hearing_date']] = df_test[['ticket_issued_date','hearing_date']].apply(pd.to_datetime)
    
int_df = pd.concat([df_train, df_test], ignore_index=True)[['ticket_issued_date','hearing_date']].dropna()
avg_date_gap = (int_df['hearing_date']-int_df['ticket_issued_date']).mean().days

df_train['date_gap'] = df_train['hearing_date']-df_train['ticket_issued_date']
df_test['date_gap'] = df_test['hearing_date']-df_test['ticket_issued_date']

df_train.loc[df_train['hearing_date'].isnull(),'date_gap'] = avg_date_gap
df_test.loc[df_test['hearing_date'].isnull(),'date_gap'] = avg_date_gap

df_train.drop(['hearing_date','ticket_issued_date'], axis=1,inplace=True)
df_test.drop(['hearing_date','ticket_issued_date'], axis=1,inplace=True)

df_train['date_gap'] = pd.to_timedelta(df_train['date_gap']).dt.days.astype('int16')
df_test['date_gap'] = pd.to_timedelta(df_test['date_gap']).dt.days.astype('int16')

For the classification model to be able to process the strings I will convert these to classes instead.

In [36]:
# Convert string values to categories need to convert agency_name, city, violation_code and disposition
#print(df_train.dtypes)

list_convert = ['agency_name', 'violation_code', 'disposition', 'inspector_name']
for col in list_convert:
    dict_list = {x: i for i, x in enumerate(pd.concat([df_train,df_test], ignore_index=True)[col].unique())}
    df_train[col] = [dict_list[x] for x in df_train[col]]
    df_test[col] = [dict_list[x] for x in df_test[col]]

At this point the data is ready for the classification.
The classification model chosen here is the Gradien Boosting Classifier.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(['compliance'], axis=1), df_train['compliance'], random_state = 0)

clf = GradientBoostingClassifier()
grid_values = {'n_estimators': [10, 50], 'learning_rate': [0.1,1], 'max_depth': [7]} # Not more for computational cost

In [38]:
grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring='roc_auc')

grid_clf.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 1], 'max_depth': [7],
                         'n_estimators': [10, 50]},
             scoring='roc_auc')

In [39]:
y_decision_scores = grid_clf.decision_function(X_test)

print('Test set AUC: ', roc_auc_score(y_test, y_decision_scores))
print('Grid best parameter (max. AUC): ', grid_clf.best_params_)
print('Grid best score (AUC): ', grid_clf.best_score_)

Test set AUC:  0.8376045589947625
Grid best parameter (max. AUC):  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50}
Grid best score (AUC):  0.8344513902641933


In [41]:
grid_clf.fit(df_train.drop(['compliance'], axis=1), df_train['compliance'])



GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 1], 'max_depth': [7],
                         'n_estimators': [10, 50]},
             scoring='roc_auc')

In [43]:
y_decision_scores_test = grid_clf.predict_proba(df_test)

final_pred = pd.DataFrame(y_decision_scores_test[:,1],index=df_test['ticket_id'])
for entry in list_non_compliant_test:
    final_pred.loc[entry] = 0

print(len(final_pred))

print(final_pred)

61001
                  0
ticket_id          
284932     0.075345
285362     0.036131
285361     0.080321
285338     0.090308
285346     0.095118
...             ...
355405     0.000000
356718     0.000000
360292     0.000000
364067     0.000000
365224     0.000000

[61001 rows x 1 columns]
